# Basic functions

In [ ]:
from math import comb
from itertools import combinations, product
import pandas as pd
import time
import sympy

In [ ]:
# calculating the run list of (0)strx(1)
def runstring(strx):
    x = "0"+strx+"1"
    r_list = [0]
    rank = 0
    for i in range(1, len(x)):
        if x[i] != x[i-1]:
            rank += 1
        r_list.append(rank)
    return r_list

def f1(strx):
    sum_f1 = 0
    for i in range(len(strx)):
        if strx[i] == "1":
            sum_f1 += (i + 1)
    return sum_f1

def f2(strx):
    sum_f2 = 0
    for i in range(len(strx)):
        if strx[i] == "1":
            sum_f2 += comb(i + 1, 2)
    return sum_f2

# Calculate number of new runs of x, a 2-insertion superstring on y
def count_new_runs_x_y(x, y):
    r_x = runstring(x)
    r_y = runstring(y)
    return r_x[-1] - r_y[-1]


# Method 1: (f1, f2, #newruns)

In [ ]:
# Given n, create a list of y's of all the strings of length n-2

#  Input: n-2 value
# Output: a list of all length n-2 strings
def generate_binary_strings(n_minus_2):
    return [''.join(seq) for seq in product('01', repeat=n_minus_2)]

In [ ]:
# Insert Characters to Form Length n strings

#  Input: string y
# Output: list of all possible 2 insertion on y
def generate_2_insertions(y, char='01'):
    n = len(y) + 2
    inserted_strings = set()  # Use a set to avoid duplicates
    
    for i in range(n):
        for j in range(i + 1, n):
            for c1 in char:
                for c2 in char:
                    inserted_str = y[:i] + c1 + y[i:j-1] + c2 + y[j-1:]
                    inserted_strings.add(inserted_str)
                    
    return list(inserted_strings)

In [ ]:
def test_2_deletion_f1_f2_ONE(n):
    start_time = time.time()  # Start the timer
    
    # Generate all possible binary strings of length n-2
    y_s = generate_binary_strings(n - 2)
    max_consecutive_duplicates_global = 0
    sequence_with_max_duplicates = []  # To store the sequences of consecutive terms with maximum duplicates

    # For each y in y_s, generate all possible x' of length n by 2 insertions
    for y in y_s:
        x_primes = generate_2_insertions(y)
        
        results = []
        for x_prime in x_primes:
            f1r_value = f1(x_prime)
            f2r_value = f2(x_prime)
            new_runs = count_new_runs_x_y(x_prime, y)
            
            results.append({
                'y': y,
                'x_prime': x_prime,
                'f1r': f1r_value,
                'f2r': f2r_value,
                'new_runs': new_runs
            })
        
        # Create DataFrame for the current y's results
        df_y = pd.DataFrame(results)
        
        # Sort the DataFrame by the tuple (f1r, f2r, new_runs)
        df_y_sorted = df_y.sort_values(by=['f1r', 'f2r', 'new_runs'], ascending=[True, True, True])
        
        # Check for consecutive duplicates and track sequences with max duplicates
        current_count = 1
        current_consecutive_terms = [df_y_sorted.iloc[0]]  # Initialize with the first term
        
        for i in range(1, len(df_y_sorted)):
            if (
                df_y_sorted.iloc[i]['f1r'] == df_y_sorted.iloc[i-1]['f1r'] and
                df_y_sorted.iloc[i]['f2r'] == df_y_sorted.iloc[i-1]['f2r'] and
                df_y_sorted.iloc[i]['new_runs'] == df_y_sorted.iloc[i-1]['new_runs']
            ):
                current_count += 1
                current_consecutive_terms.append(df_y_sorted.iloc[i])
            else:
                if current_count > max_consecutive_duplicates_global:
                    max_consecutive_duplicates_global = current_count
                    sequence_with_max_duplicates = [
                        [
                            current_consecutive_terms[0]['y'],
                            [term['x_prime'] for term in current_consecutive_terms],
                            current_consecutive_terms[0]['f1r'],
                            current_consecutive_terms[0]['f2r'],
                            current_consecutive_terms[0]['new_runs']
                        ]
                    ]
                elif current_count == max_consecutive_duplicates_global:
                    sequence_with_max_duplicates.append(
                        [
                            current_consecutive_terms[0]['y'],
                            [term['x_prime'] for term in current_consecutive_terms],
                            current_consecutive_terms[0]['f1r'],
                            current_consecutive_terms[0]['f2r'],
                            current_consecutive_terms[0]['new_runs']
                        ]
                    )
                # Reset for the next sequence
                current_count = 1
                current_consecutive_terms = [df_y_sorted.iloc[i]]
        
        # Final check after loop
        if current_count > max_consecutive_duplicates_global:
            max_consecutive_duplicates_global = current_count
            sequence_with_max_duplicates = [
                [
                    current_consecutive_terms[0]['y'],
                    [term['x_prime'] for term in current_consecutive_terms],
                    current_consecutive_terms[0]['f1r'],
                    current_consecutive_terms[0]['f2r'],
                    current_consecutive_terms[0]['new_runs']
                ]
            ]
        elif current_count == max_consecutive_duplicates_global:
            sequence_with_max_duplicates.append(
                [
                    current_consecutive_terms[0]['y'],
                    [term['x_prime'] for term in current_consecutive_terms],
                    current_consecutive_terms[0]['f1r'],
                    current_consecutive_terms[0]['f2r'],
                    current_consecutive_terms[0]['new_runs']
                ]
            )
    end_time = time.time()  # End the timer
    runtime = end_time - start_time  # Calculate the runtime
    print(f"Time: {runtime:.2f} seconds")

    print(f"For n={n}, Maximum consecutive duplicates among all y's: {max_consecutive_duplicates_global}\n")
    for seq in sequence_with_max_duplicates:
        print(f"y = {seq[0]},  2-Insertion:{seq[1]},  f1 = {seq[2]}, f2 = {seq[3]}, NewRuns = {seq[4]}")
    

    
    # Return the maximum number of consecutive duplicates and the sequences that achieve this max
    # return max_consecutive_duplicates_global, sequence_with_max_duplicates


In [ ]:
test_2_deletion_f1_f2_ONE(10)

In [ ]:
test_2_deletion_f1_f2_ONE(11)

In [ ]:
test_2_deletion_f1_f2_ONE(12)

In [ ]:
test_2_deletion_f1_f2_ONE(13)

In [ ]:
test_2_deletion_f1_f2_ONE(14)

In [ ]:
# checking:
y = "000001111100"
x1 = '00000111101010'
x2 = '10100001111100'
x3 = '00001010111100'
print(f1(x1), f1(x2), f1(x3), f2(x1), f2(x2), f2(x3))

# Method 2: using (f1 mod p, f2 mod p, #), but skip since 1 doesn't work